paanmego@gmail.com Maded by 2024.09

In [2]:
import os
import torch
import shutil
import logging
import warnings
import numpy as np
import pandas as pd

from collections import Counter
import re

from tqdm import tqdm
from rouge import Rouge
from datasets import Dataset as HFDataset
from torch.utils.data import Dataset, DataLoader
from transformers import T5TokenizerFast, AutoConfig
from transformers import (T5ForConditionalGeneration, T5TokenizerFast, 
                          get_cosine_schedule_with_warmup)
from typing import List, Dict, Any
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
import wandb
import torch.distributed as dist
import torch.cuda.amp as amp

In [3]:
config = {
    "general": {
        "data_path": "../data/",
        "model_name": "lcw99/t5-large-korean-text-summary",
        "output_dir": "./results"
    },
    "tokenizer": {
        "encoder_max_len": 1024,
        "decoder_max_len": 512,
        "special_tokens": ['#Person1#', '#Person#', '#Person4#', '#CarNumber#', '#Person2#', '#SSN#', '#Person6#',
                            '#DateOfBirth#', '#Email#', '#PhoneNumber#', '#Address#', '#Person3#', '#CardNumber#',
                            '#PassportNumber#', '#Person5#', '#Person7#'],
        'preserve_special_tokens': True,
    },
    "training": {
        "num_train_epochs": 5,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 2,
        "per_device_eval_batch_size": 2,
        "warmup_steps": 500,
        "weight_decay": 0.01,
        "logging_steps": 3000,
        "eval_steps": 3000,
        "save_steps": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "gradient_accumulation_steps": 2,
        "generation_max_length": 256,
        #"early_stopping_patience": 3,
        #"early_stopping_threshold": 0.001,
        "max_grad_norm": 1.0,
    },
    "inference": {
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 512, #256
        "num_beams": 11, #4
        "batch_size": 1,
        "remove_tokens": ['<usr>', '</s>', '<s>', '<pad>'],
        "ckt_path": "./results/final_t5_model.pt"
    }
}

In [4]:
# 경고 메시지 무시 및 로깅 설정
warnings.filterwarnings("ignore", message=".*resume_download.*", category=FutureWarning)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

file_handler = logging.FileHandler('training.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(file_handler)

# CUDA 설정 : 성능 최적화
torch.backends.cudnn.benchmark = True
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
# Wandb 초기화 : 자신에 맞게 수정
wandb.init(
    entity="dl12",
    project="lm",
    name="lcw99/t5-large-final",
)

2024-09-10 11:41:49,590 - ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: paanmego (dl12). Use `wandb login --relogin` to force relogin


In [6]:
# 오래된 로그 파일 삭제
def clear_logs():
    log_file = 'training.log'
    if os.path.exists(log_file):
        os.remove(log_file)
    logger.info("Training log cleared.")

# 오래된 체크포인트 자동 삭제(n개만 유지)
def cleanup_old_checkpoints(config: Dict[str, Any], keep_last_n: int = 3):
    checkpoint_dir = os.path.join(config['general']['output_dir'], "checkpoints")
    checkpoint_files = sorted([f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint_") and f.endswith(".pt")], reverse=True)
    for checkpoint in checkpoint_files[keep_last_n:]:
        os.remove(os.path.join(checkpoint_dir, checkpoint))
        logger.info(f"Removed old checkpoint: {checkpoint}")

# 시퀀스를 작은 청크로 나누는 함수
def chunk_sequence(sequence, chunk_size):
    return [sequence[i:i + chunk_size] for i in range(0, len(sequence), chunk_size)]

class Preprocess:
    def __init__(self, tokenizer: T5TokenizerFast):
        self.tokenizer = tokenizer
        self.bos_token = tokenizer.bos_token or ''
        self.eos_token = tokenizer.eos_token or ''
        self.chunk_size = 512

    @staticmethod
    def make_set_as_df(file_path: str, is_train: bool = True) -> pd.DataFrame:
        df = pd.read_csv(file_path)
        
        replacements = {
            'ㅋㅋ': '웃기다', 'ㅇ로': '으로', '제ㅏ': '제가', 'ㅍ알': ' 알', 'ㄷ거': '거',
            '##': '#', '회사 #에서': '회사에서',
            '#작은': '#Person2#: 작은', '#여기서': '#Person1#: 여기서', '#나': '#Person2#: 나',
            '#페리에와': '#Person1#: 페리에와', '#샐러드용': '#Person1#: 샐러드용',
            '#어디': '#Person1#: 어디', '#잠깐만요': '#Person1#: 잠깐만요',
            '#하지만': '#Person1#: 하지만', '#사람1만기': '#Person1#: 만기',
            '#PhoneNumber이고': '#PhoneNumber#이고', '#Person1:': '#Person1#:',
            '#Person2:': '#Person2#:', '#Person#': '#Person2#:', '사람1#:': '#Person1#:',
            '#고객님:': '#Person2#: 고객님', '선생님: ': '', '로저스 씨: ': '',
            '남자: 아악.': '', '남자: 고마워.': ''
        }
        
        df['dialogue'] = df['dialogue'].replace(replacements, regex=True)

        if 'summary' in df.columns:
            summary_replacements = {
                '사람1#': '#Person1#', '사람2#': '#Person2#', '#사람1#': '#Person1#'
            }
            df['summary'] = df['summary'].replace(summary_replacements, regex=True)

        if is_train:
            return df[['fname', 'dialogue', 'summary', 'topic']]
        else:
            return df[['fname', 'dialogue']]

    # 모델 입력 생성 : 대화를 청크로 나눔, 각 청크에 대해 프롬프트 생성
    def make_input(self, dataset: pd.DataFrame, is_test: bool = False):
        encoder_input, decoder_input, decoder_output = [], [], []

        for _, row in dataset.iterrows():
            dialogue = str(row['dialogue'])
            summary = str(row['summary']) if not is_test else ""
            
            for chunk in [dialogue[i:i+self.chunk_size] for i in range(0, len(dialogue), self.chunk_size)]:
                encoder_input.append(create_prompt(chunk))
                if not is_test:
                    decoder_input.append(self.bos_token + summary)
                    decoder_output.append(summary + self.eos_token)
                else:
                    decoder_input.append(self.bos_token)
                    decoder_output.append("")

        return (encoder_input, decoder_input, decoder_output) if not is_test else (encoder_input, decoder_input)


In [9]:
# TS 모델을 상속받아 긴 시퀀스를 처리하도록 확장
class LongformerEncoderDecoderForConditionalGeneration(T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)

# 배치 내의 시퀀스 길이를 동적으로 패딩
class DynamicPaddingCollator:
    def __init__(self, pad_token_id: int, label_pad_token_id: int):
        self.pad_token_id = pad_token_id
        self.label_pad_token_id = label_pad_token_id

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        if all(len(x["input_ids"]) == len(features[0]["input_ids"]) for x in features):
            return self.stack_tensors(features)
        else:
            return self.pad_tensors(features)
    
    # 모든 텐서를 스택
    def stack_tensors(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
    # 텐서 크기가 동일한 경우만 스택
        try:
            return {k: torch.stack([torch.tensor(f[k]) if isinstance(f[k], list) else f[k] for f in features]) for k in features[0].keys()}
        except RuntimeError as e:
            # 크기가 다른 텐서가 있는 경우 예외 처리
            if "stack expects each tensor to be equal size" in str(e):
                return self.pad_tensors(features)
            else:
                raise e    
            
    # 가장 긴 시퀀스에 맞춰 나머지 시퀀스를 패딩
    def pad_tensors(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        padded_features = {}
        for key in features[0].keys():
            padding_value = self.label_pad_token_id if key == "labels" else self.pad_token_id
            # 리스트를 텐서로 변환하여 패딩 처리
            padded_features[key] = torch.nn.utils.rnn.pad_sequence(
                [torch.tensor(f[key]) if isinstance(f[key], list) else f[key] for f in features], 
                batch_first=True, 
                padding_value=padding_value
            )
        return padded_features

def create_optimized_dataloaders(config, tokenizer, preprocess):
    train_df = preprocess.make_set_as_df(os.path.join(config['general']['data_path'], 'train.csv'), is_train=True)
    val_df = preprocess.make_set_as_df(os.path.join(config['general']['data_path'], 'dev.csv'), is_train=True)

    train_encoder_input, train_decoder_input, train_decoder_output = preprocess.make_input(train_df, is_test=False)
    val_encoder_input, val_decoder_input, val_decoder_output = preprocess.make_input(val_df, is_test=False)

    train_dataset = HFDataset.from_dict({
        "encoder_input": train_encoder_input,
        "decoder_input": train_decoder_input,
        "decoder_output": train_decoder_output
    })

    val_dataset = HFDataset.from_dict({
        "encoder_input": val_encoder_input,
        "decoder_input": val_decoder_input,
        "decoder_output": val_decoder_output
    })

    # 입력과 레이블을 토큰화
    def preprocess_function(examples):
        model_inputs = tokenizer(examples["encoder_input"], max_length=config['tokenizer']['encoder_max_len'], truncation=True)
        labels = tokenizer(text_target=examples["decoder_output"], max_length=config['tokenizer']['decoder_max_len'], truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    train_dataset = train_dataset.map(preprocess_function, batched=True, num_proc=4, remove_columns=train_dataset.column_names)
    val_dataset = val_dataset.map(preprocess_function, batched=True, num_proc=4, remove_columns=val_dataset.column_names)

    # 분산 학습을 위한 샘플러 설정
    train_sampler = DistributedSampler(train_dataset) if dist.is_initialized() else None
    val_sampler = DistributedSampler(val_dataset, shuffle=False) if dist.is_initialized() else None

    collator = DynamicPaddingCollator(pad_token_id=tokenizer.pad_token_id, label_pad_token_id=tokenizer.pad_token_id)

    train_dataloader = DataLoader(train_dataset, batch_size=config['training']['per_device_train_batch_size'], 
                                  sampler=train_sampler, collate_fn=collator, pin_memory=True, num_workers=4)
    eval_dataloader = DataLoader(val_dataset, batch_size=config['training']['per_device_eval_batch_size'], 
                                 sampler=val_sampler, collate_fn=collator, pin_memory=True, num_workers=4)
    
    return train_dataloader, eval_dataloader

def save_checkpoint(model, optimizer, epoch, step, loss, config, is_best=False):
    checkpoint_dir = os.path.join(config['general']['output_dir'], "checkpoints")
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_step_{step}.pt")
    
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_state_dict = model.module.state_dict()
    else:
        model_state_dict = model.state_dict()
    
    torch.save({
        'model_state_dict': model_state_dict,
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'step': step,
        'loss': loss
    }, checkpoint_path)
    
    if is_best:
        best_model_path = os.path.join(config['general']['output_dir'], "best_t5_model.pt")
        shutil.copyfile(checkpoint_path, best_model_path)
    
    logger.info(f"Checkpoint saved at {checkpoint_path}")
    return checkpoint_path

def load_tokenizer(config):
    tokenizer_path = os.path.join(config['general']['output_dir'], "final_t5_tokenizer")
    tokenizer = T5TokenizerFast.from_pretrained(tokenizer_path)
    logger.info(f"Tokenizer loaded from {tokenizer_path}")
    return tokenizer

def save_tokenizer(tokenizer, output_dir):
    tokenizer_save_path = os.path.join(output_dir, "final_t5_tokenizer")
    tokenizer.save_pretrained(tokenizer_save_path)
    logger.info(f"Final tokenizer saved at {tokenizer_save_path}")

def load_tokenizer_and_model(config, device, for_inference=False):
    logger.info(f"{'Loading tokenizer & model for inference' if for_inference else 'Loading tokenizer & model for training'}")
    model_name = config['general']['model_name']
    
    if for_inference:
        tokenizer = load_tokenizer(config)
        model = LongformerEncoderDecoderForConditionalGeneration.from_pretrained(
            os.path.join(config['general']['output_dir'], "final_t5_model")
        )
    else:
        tokenizer = T5TokenizerFast.from_pretrained(model_name)
        special_tokens = config['tokenizer'].get('special_tokens', [])
        if special_tokens:
            special_tokens_dict = {'additional_special_tokens': special_tokens}
            num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
            logger.info(f"Added {num_added_toks} special tokens: {special_tokens}")

        model = LongformerEncoderDecoderForConditionalGeneration.from_pretrained(model_name)
        
        if num_added_toks > 0:
            model.resize_token_embeddings(len(tokenizer))
            logger.info(f"Model embeddings resized to accommodate {num_added_toks} new tokens")
    
    tokenizer.do_not_trim_special_tokens = True
    
    model.to(device)
    return model, tokenizer


In [8]:
def load_tokenizer_and_model(config, device, for_inference=False, checkpoint_path=None):
    logger.info(f"{'Loading tokenizer & model for inference' if for_inference else 'Loading tokenizer & model for training'}")
    model_name = config['general']['model_name']
    
    if for_inference:
        tokenizer = load_tokenizer(config)
        
        if checkpoint_path:
            logger.info(f"Loading model from checkpoint: {checkpoint_path}")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            
            # 체크포인트에서 config를 로드
            config = checkpoint.get('config', config)
            
            # 체크포인트의 vocab_size를 사용하여 모델 초기화
            model_config = AutoConfig.from_pretrained(model_name)
            model_config.vocab_size = checkpoint['model_state_dict']['shared.weight'].shape[0]
            
            model = LongformerEncoderDecoderForConditionalGeneration(model_config)
            
            # 모델의 state_dict를 직접 로드
            model.load_state_dict(checkpoint['model_state_dict'])
            
            # tokenizer의 vocab도 업데이트
            tokenizer.resize_token_embeddings(model_config.vocab_size)
        else:
            logger.info(f"Loading model from final T5 model directory")
            model = LongformerEncoderDecoderForConditionalGeneration.from_pretrained(
                os.path.join(config['general']['output_dir'], "final_t5_model")
            )
    else:
        tokenizer = T5TokenizerFast.from_pretrained(model_name)
        special_tokens = config['tokenizer'].get('special_tokens', [])
        if special_tokens:
            special_tokens_dict = {'additional_special_tokens': special_tokens}
            num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
            logger.info(f"Added {num_added_toks} special tokens: {special_tokens}")

        model = LongformerEncoderDecoderForConditionalGeneration.from_pretrained(model_name)
        
        if 'num_added_toks' in locals() and num_added_toks > 0:
            model.resize_token_embeddings(len(tokenizer))
            logger.info(f"Model embeddings resized to accommodate {num_added_toks} new tokens")
    
    tokenizer.do_not_trim_special_tokens = True
    
    model.to(device)
    return model, tokenizer

In [10]:
# 성능 평가 함수
def evaluate(model: torch.nn.Module, dataloader: DataLoader, tokenizer: T5TokenizerFast, device: torch.device, config: Dict[str, Any]):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", dynamic_ncols=True, ascii=True):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += outputs.loss.item()

            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=config['training']['generation_max_length'],
                num_beams=4,
                early_stopping=True,
                no_repeat_ngram_size=3,
                length_penalty=1.0,
                min_length=10,
                use_cache=True
            )
            
            decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            
            all_preds.extend([pred.strip() for pred in decoded_preds])
            all_labels.extend([label.strip() for label in decoded_labels])
            
            del input_ids, attention_mask, labels, outputs, generated_ids
            torch.cuda.empty_cache()
    
     # 평균 손실 계산
    avg_loss = total_loss / len(dataloader)

     # ROUGE 점수 계산
    rouge = Rouge()
    if all_preds and all_labels:
        try:
            scores = rouge.get_scores(all_preds, all_labels, avg=True)
        except ValueError as e:
            logger.error(f"Error in ROUGE calculation: {str(e)}")
            logger.info(f"Sample predictions: {all_preds[:5]}")
            logger.info(f"Sample labels: {all_labels[:5]}")
            scores = {'rouge-1': {'f': 0.0}, 'rouge-2': {'f': 0.0}, 'rouge-l': {'f': 0.0}}
    else:
        logger.warning("No valid predictions or labels for ROUGE calculation.")
        scores = {'rouge-1': {'f': 0.0}, 'rouge-2': {'f': 0.0}, 'rouge-l': {'f': 0.0}}
    
    # 결과 로깅
    logger.info(f"Evaluation Loss: {avg_loss:.4f}")
    logger.info("ROUGE Scores:")
    for k, v in scores.items():
        logger.info(f"  {k}: {v['f']:.4f}")
    
    return avg_loss, scores

# 특수 토큰 제거
def remove_special_tokens(text, remove_tokens):
    for token in remove_tokens:
        text = text.replace(token, '')
    return text

def clean_summary(text):
    text = re.sub(r'#Person\d*#', lambda m: f'PERSONTAGPLACEHOLDER{m.group()}PERSONTAGPLACEHOLDER', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'PERSONTAGPLACEHOLDER(#Person\d*#)PERSONTAGPLACEHOLDER', r'\1', text)
    text = re.sub(r'(#Person\d*#)(\s*\1)+', r'\1', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'(#Person\d*#)\s', r'\1', text)
    return text.strip()

# 모델 입력용 프롬프트 생성
def create_prompt(dialogue):
    # Basic
    prompt = f"summarize: {dialogue}"
    return prompt

In [11]:
# 모델학습의 전체 과정 관리 함수
def train_and_save(config):
    if torch.cuda.device_count() > 1:
        dist.init_process_group(backend='nccl')
        local_rank = dist.get_rank()
        device = torch.device(f'cuda:{local_rank}')
    else:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model, tokenizer = load_tokenizer_and_model(config, device)
    if torch.cuda.device_count() > 1:
        model = DDP(model, device_ids=[local_rank], output_device=local_rank)
    
    preprocess = Preprocess(tokenizer)
    train_dataloader, eval_dataloader = create_optimized_dataloaders(config, tokenizer, preprocess)

    optimizer = torch.optim.AdamW(model.parameters(), lr=config['training']['learning_rate'], weight_decay=config['training']['weight_decay'])
    num_training_steps = len(train_dataloader) * config['training']['num_train_epochs']
    lr_scheduler = get_cosine_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=config['training']['warmup_steps'], num_training_steps=num_training_steps)
    scaler = torch.amp.GradScaler()
    #early_stopping_callback = EarlyStoppingCallback(patience=config['training']['early_stopping_patience'], threshold=config['training']['early_stopping_threshold'])

    best_eval_loss = float('inf')
    best_model_path = None

    for epoch in range(config['training']['num_train_epochs']):
        model.train()
        total_loss = 0
        for step, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1}", dynamic_ncols=True, ascii=True)):
            batch = {k: v.to(device) for k, v in batch.items()}
            
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(**batch)
                loss = outputs.loss
            
            total_loss += loss.item()
            scaler.scale(loss).backward()

            if (step + 1) % config['training']['gradient_accumulation_steps'] == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config['training']['max_grad_norm'])
                scaler.step(optimizer)
                scaler.update()
                lr_scheduler.step()
                optimizer.zero_grad()

            if (not dist.is_initialized() or dist.get_rank() == 0) and step % config['training']['logging_steps'] == 0:
                logger.info(f"Epoch {epoch+1}, Step {step}: Loss {loss.item():.4f}")
                wandb.log({"train_loss": loss.item(), "epoch": epoch + 1, "step": step})
        
        avg_train_loss = total_loss / len(train_dataloader)

        if (not dist.is_initialized() or dist.get_rank() == 0) and config['training']['save_steps'] == 'epoch':
                save_checkpoint(model, optimizer, epoch, step, loss.item(), config)
                cleanup_old_checkpoints(config, keep_last_n=config['training']['save_total_limit'])
        
        if not dist.is_initialized() or dist.get_rank() == 0:
            logger.info(f"Epoch {epoch+1} - Average train loss: {avg_train_loss:.4f}")
            wandb.log({"avg_train_loss": avg_train_loss, "epoch": epoch + 1})
        
        eval_loss, rouge_scores = evaluate(model, eval_dataloader, tokenizer, device, config)
        if not dist.is_initialized() or dist.get_rank() == 0:
            logger.info(f"Epoch {epoch+1} - Eval Loss: {eval_loss:.4f}")
            logger.info(f"Epoch {epoch+1} - Rouge Scores:")
            for k, v in rouge_scores.items():
                logger.info(f"  {k.upper()}: {v['f']:.4f}")
        
            wandb.log({
                "eval_loss": eval_loss,
                "rouge-1_f": rouge_scores['rouge-1']['f'],
                "rouge-2_f": rouge_scores['rouge-2']['f'],
                "rouge-l_f": rouge_scores['rouge-l']['f'],
                "epoch": epoch + 1
            })

        if eval_loss < best_eval_loss:
            best_eval_loss = eval_loss
            best_model_path = save_checkpoint(model, optimizer, epoch, -1, eval_loss, config, is_best=True)
            logger.info(f"New best model saved with eval loss: {eval_loss:.4f}")
        else:
            logger.info(f"No improvement in eval loss. Current best: {best_eval_loss:.4f}")

        # if early_stopping_callback(eval_loss, epoch):
        #     logger.info(f"Early stopping triggered at epoch {epoch + 1}")
        #     if best_model_path:
        #         logger.info(f"Loading best model from {best_model_path}")
        #         checkpoint = torch.load(best_model_path)
        #         if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        #             model.module.load_state_dict(checkpoint['model_state_dict'])
        #         else:
        #             model.load_state_dict(checkpoint['model_state_dict'])
        #     break
    
    # 최종 모델 저장 (항상 best model 저장)
    final_model_path = os.path.join(config['general']['output_dir'], "final_t5_model")
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model.module.save_pretrained(final_model_path)
    else:
        model.save_pretrained(final_model_path)
    logger.info(f"Final model (best performing) saved at {final_model_path}")
    
    # 학습이 끝난 후 토크나이저 저장
    save_tokenizer(tokenizer, config['general']['output_dir'])

    wandb.finish()
    # 로그 파일 지우기
    clear_logs()

In [12]:
if torch.cuda.device_count() > 1:
        dist.init_process_group(backend='nccl')
        torch.cuda.set_device(dist.get_rank())
    
logger.info("Starting training...")
max_retries = 3
retry_count = 0

# CUDA Out of Memory Exception 처리 (계속 에러가 나는 경우 kill -9 로 백그라운 파이썬 강제 처리 해야함)
while retry_count < max_retries:
    try:
        train_and_save(config)
        break
    except RuntimeError as e:
        if "out of memory" in str(e):
            retry_count += 1
            logger.warning(f"CUDA out of memory error occurred. Attempt {retry_count} of {max_retries}.")
            torch.cuda.empty_cache()
            if retry_count == max_retries:
                logger.error("Max retries reached. Exiting.")
                raise
        else:
            logger.error("Unexpected error occurred.", exc_info=True)
            raise

2024-09-10 11:42:35,627 - INFO - Starting training...
2024-09-10 11:42:35,660 - INFO - Loading tokenizer & model for training
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-09-10 11:42:36,132 - INFO - Added 16 special tokens: ['#Person1#', '#Person#', '#Person4#', '#CarNumber#', '#Person2#', '#SSN#', '#Person6#', '#DateOfBirth#', '#Email#', '#PhoneNumber#', '#Address#', '#Person3#', '#CardNumber#', '#PassportNumber#', '#Person5#', '#Person7#']
2024-09-10 11:42:37,951 - INFO - Model embeddings resized to accommodate 16 new tokens


Map (num_proc=4):   0%|          | 0/16306 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/647 [00:00<?, ? examples/s]

Epoch 1: 100%|##########| 8153/8153 [31:33<00:00,  4.31it/s]
2024-09-10 12:14:31,260 - INFO - Checkpoint saved at ./results/checkpoints/checkpoint_epoch_0_step_8152.pt
2024-09-10 12:14:31,261 - INFO - Epoch 1 - Average train loss: 1.5444
Evaluating: 100%|##########| 324/324 [07:45<00:00,  1.44s/it]
2024-09-10 12:22:17,124 - INFO - Evaluation Loss: 1.0756
2024-09-10 12:22:17,124 - INFO - ROUGE Scores:
2024-09-10 12:22:17,125 - INFO -   rouge-1: 0.2510
2024-09-10 12:22:17,126 - INFO -   rouge-2: 0.0860
2024-09-10 12:22:17,126 - INFO -   rouge-l: 0.2406
2024-09-10 12:22:17,127 - INFO - Epoch 1 - Eval Loss: 1.0756
2024-09-10 12:22:17,128 - INFO - Epoch 1 - Rouge Scores:
2024-09-10 12:22:17,128 - INFO -   ROUGE-1: 0.2510
2024-09-10 12:22:17,129 - INFO -   ROUGE-2: 0.0860
2024-09-10 12:22:17,129 - INFO -   ROUGE-L: 0.2406
2024-09-10 12:22:37,790 - INFO - Checkpoint saved at ./results/checkpoints/checkpoint_epoch_0_step_-1.pt
2024-09-10 12:22:37,791 - INFO - New best model saved with eval los

In [28]:
# 테스트 데이터 적용 함수
def inference(config, model, tokenizer, preprocessor):
    test_data = preprocessor.make_set_as_df(os.path.join(config['general']['data_path'], 'test.csv'), is_train=False)
    fnames = test_data['fname'].tolist()
    test_data.set_index('fname', inplace=True)
    
    encoder_input_test = test_data['dialogue'].apply(lambda x: create_prompt(x)).tolist()
    
    inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                       add_special_tokens=True, truncation=True,
                       max_length=config['tokenizer']['encoder_max_len'])
    
    dataset = list(zip(inputs['input_ids'], inputs['attention_mask'], fnames))
    
    def collate_fn(batch):
        input_ids, attention_masks, fnames = zip(*batch)
        return torch.stack(input_ids), torch.stack(attention_masks), fnames

    dataloader = DataLoader(dataset, batch_size=config['inference']['batch_size'], 
                            num_workers=4, pin_memory=True, collate_fn=collate_fn)

    model.eval()
    summary = []
    text_ids = []
    
    with torch.no_grad():
        for input_ids, attention_mask, ids in tqdm(dataloader, desc="Inference", dynamic_ncols=True, ascii=True):
            text_ids.extend(ids)
            
            input_ids = input_ids.to(model.device, non_blocking=True)
            attention_mask = attention_mask.to(model.device, non_blocking=True)
            
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                early_stopping=config['inference']['early_stopping'],
                max_length=config['inference']['generate_max_length'],
                #min_length=config['inference']['min_length'],
                num_beams=config['inference']['num_beams'],
                # length_penalty=1.0,
                # repetition_penalty=1.1,
                do_sample=False,
                #temperature=0.3,
                #top_k=50,
                #top_p=0.9,
                output_scores=True,
                return_dict_in_generate=True,
            )
            
            decoded_summaries = tokenizer.batch_decode(generated_ids.sequences, skip_special_tokens=False)
            processed_summaries = [clean_summary(summary) for summary in decoded_summaries]
            summary.extend(processed_summaries)
    
    output = pd.DataFrame({"fname": text_ids, "summary": summary})

    output_file = os.path.join(config['inference']['result_path'], "inference_output.csv")
    os.makedirs(config['inference']['result_path'], exist_ok=True)
    output.to_csv(output_file, index=False)
    logger.info(f"Inference results saved to: {output_file}")

    return output

In [18]:
logger.info("Starting inference...")
try:
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model, tokenizer = load_tokenizer_and_model(config, device, for_inference=True)
    #model, tokenizer = load_tokenizer_and_model(config, device, for_inference=True, checkpoint_path="./results/checkpoints/checkpoint_epoch_4_step_8152.pt")
    preprocessor = Preprocess(tokenizer)
    output = inference(config, model, tokenizer, preprocessor)
    logger.info(f"Inference completed. Output saved to: {os.path.join(config['inference']['result_path'], 'inference_output.csv')}")
except FileNotFoundError as e:
    logger.error(f"Model file not found: {str(e)}")
except Exception as e:
    logger.error(f"Error occurred during inference: {str(e)}", exc_info=True)

2024-09-10 11:10:26,366 - INFO - Starting inference...
2024-09-10 11:10:26,368 - INFO - Loading tokenizer & model for inference
2024-09-10 11:10:26,427 - INFO - Tokenizer loaded from ./results/final_t5_tokenizer
Inference: 100%|##########| 499/499 [12:23<00:00,  1.49s/it]
2024-09-10 11:22:52,230 - INFO - Inference results saved to: ./prediction/inference_output.csv
2024-09-10 11:22:52,249 - INFO - Inference completed. Output saved to: ./prediction/inference_output.csv
